<a href="https://colab.research.google.com/github/alisonsoong/NASA-SEES-Internship-2021/blob/main/SEES_2021_ARSET_GEE_Training_Session_1_js_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_earthengine.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_earthengine.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Import Earth Engine API and authenticate<a class="anchor" id="import-api"></a>

The Earth Engine API is installed by default in Google Colaboratory. The following steps must be completed

*   for each new Colab session,
*   if you restart your Colab kernel, or 
*   if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the Google Earth Engine API into your session.

In [1]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [2]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=E7jSgdWsKUDkevxVPIJhrKxNnKz22M5tD9OZz_L0Ypw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjS7i6iwLxaiygrhJ-O-lhZn_xHLjJYBCZLi4ysyq59cXo9VKkCMuQ

Successfully saved authorization token.


## Install the geemap Python package

Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api and folium for map display. One dependency not installed here is ipyleaflet, since Colab sadly does not support it.

With **geemap** installed, we do not have to write our own **folium** display function.

In [3]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 


geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


# Basics Functions in GEE Colab & geemap Activity

Here the Javascript code (link on slide 23 of the ARSET Session 1) is converted into Python.

[Link to Presentation Slides PDF](http://appliedsciences.nasa.gov/sites/default/files/2021-06/GEE_Land_Part1_0.pdf)

From ARSET training **Part 1: Google Earth Engine Basics and General Applications**
*Wednesday, June 16, 2021*




## Create an map with multiple vegetation index layers

In [4]:
Map = geemap.Map()

# Define point of interest to spatially filter data. 
# ee.Geometry.Point() function is demonstrated here. 
# line 3 in js example
point = ee.Geometry.Point([-122.292, 37.9018]) 

# Import the Landsat 8 Surface Reflectance image collection.
# line 6 in js example
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Get the least cloudy image in May 2021. 
# Note use of back slashes for code legibility.
# lines 9-14 in js example
# First step is to filter spatially
image = ee.Image(l8.filterBounds(point) \
                 # next filter by date - month of May
                 .filterDate('2021-05-01', '2021-05-31') \
                 # use CLOUD_COVER metadata to sort from clearest
                 # to cloudiest pixels
                 .sort('CLOUD_COVER') \
                 # get the first values in the sorted image
                 .first())

# View selected image metadata 
# Get information about the bands as a list.
# variation on line 17 in js example
# All metadata.
print('All image metadata:')
meta = image.getInfo()
print("meta is data type", type(meta))
for k in meta.keys():
  print(k, meta[k])
print()
print('Image band names:')
band_names = image.bandNames()
# Get a list of band names
bn = band_names.getInfo()
# Use a for loop to print each value in the list
for b in bn:
  print(b)  # ee.List of band names
print()

# Select and rename bands 
# like line 20 in js example 
image = image.select(['B5','B4','B2'],['nir','red','blue']) 

# Compute the Normalized Difference Vegetation Index (NDVI). 
# NDVI = (NIR - Red) / (NIR + Red)  
# ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI') # doesn't work
# like line 25 in js example
ndvi = image.normalizedDifference(['nir','red']) 

# Display NDVI results.
# like lines 28-29 in js example 
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']} 
Map.addLayer(ndvi, ndviParams, 'NDVI image')

# Compute and display Enhanced Vegetation Index (EVI) 
# EVI = 2.5 * ((NIR - Red) / (NIR + 6 * Red – 7.5 * Blue + 1)) #\n', 
# like lines 33-42 in js example
# Compute the EVI using an expression.
evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': image.select('nir'),
        'RED': image.select('red'),
        'BLUE': image.select('blue')
    })

eviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']} 
Map.addLayer(evi, eviParams, 'EVI image')

# Compute and display Soil-Adjusted Vegetation Index (SAVI)
# SAVI = ((NIR - Red) / (NIR + Red + 0.5)) * 1.5
# like lines 46-55 in js example
savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {
        'NIR':image.select('nir'),
        'RED':image.select('red')
    }) 
saviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']} 
Map.addLayer(savi, saviParams, 'SAVI image')
Map.centerObject(point, 9) 

# Mapping an index over a collection.
# Define a function that produces index
def indices(img):
  img = img.select(['B5','B4','B2'],['nir','red','blue'])
  evi = img.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': img.select('nir'),
        'RED': img.select('red'),
        'BLUE': img.select('blue')
    }).rename('EVI')
  return img.addBands(evi)
  
collection = l8.filterBounds(point) \
   .filterDate('2021-01-01', '2021-05-31') \
   .filterMetadata('CLOUD_COVER_LAND','less_than',20) \
   .sort('CLOUD_COVER') \
   .map(algorithm=indices)

new_evi = collection.first()
 
print('evi Image band names:')
band_names = new_evi.bandNames()
# Get a list of band names
bn = band_names.getInfo()
# Use a for loop to print each value in the list
for b in bn:
  print(b)  # ee.List of band names
print()

evi2 = new_evi.select(['EVI'])
Map.addLayer(evi2, eviParams, 'EVI Collection image')
Map




All image metadata:
meta is data type <class 'dict'>
type Image
bands [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7681, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 463185, 0, -30, 4264515]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7681, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 463185, 0, -30, 4264515]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7681, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 463185, 0, -30, 4264515]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7681, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 463185, 0, -30, 4264515]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7681, 7801], 'crs': 